In [82]:
from geoalchemy2 import Geometry, load_spatialite
from flask_caching import Cache
from sqlalchemy import event, Table, insert
from sqlalchemy.orm import sessionmaker
import os
from flask import Flask
from quickannotator.db import Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation
from quickannotator.db import db
import large_image

In [83]:
def insert_project(app, db, name, description, is_dataset_large):
    with app.app_context():
        project = Project(name=name,
                          description=description,
                          is_dataset_large=is_dataset_large)
        db.session.add(project)
        db.session.commit()
        
        project_path = f"data/{project.id}"
        if not os.path.exists(project_path):
            os.mkdir(project_path)
        
def insert_image(app, db, project_id, name, path, height, width, dz_tilesize, embedding_coord, group_id, split):
    with app.app_context():
        image = Image(project_id=project_id,
                      name=name,
                      path=path,
                      height=height,
                      width=width,
                      dz_tilesize=dz_tilesize,
                      embedding_coord=embedding_coord,
                      group_id=group_id,
                      split=split)
        db.session.add(image)
        db.session.commit()
        
def insert_annotation_class(app, db, project_id, name, color, magnification, patchsize, tilesize, dl_model_objectref):
    with app.app_context():
        annotation_class = AnnotationClass(project_id=project_id,
                                           name=name,
                                           color=color,
                                           magnification=magnification,
                                           patchsize=patchsize,
                                           tilesize=tilesize,
                                           dl_model_objectref=dl_model_objectref)
        db.session.add(annotation_class)
        db.session.commit()
        
def insert_annotation(app, db, image_id, annotation_class_id, gtpred):
    with app.app_context():
        table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
        annotation = Annotation(centroid="POINT(1 1)",
                                area=1000,
                                polygon="MULTIPOLYGON(((0 0, 0 1, 1 1, 1 0, 0 0)))",
                                custom_metrics={"iou": 0.5})
        
        table = Table(table_name, db.metadata, autoload_with=db.engine)
        stmt = table.insert().values([annotation])
        db.session.execute(stmt)
        db.session.commit()
        

def create_annotation_table(db, image_id, annotation_class_id, gtpred):
    table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
    table = Annotation.__table__.to_metadata(db.metadata, name=table_name)
    db.metadata.create_all(bind=db.engine, tables=[table])
    
        

In [84]:
app = Flask("app")
SearchCache = Cache(config={'CACHE_TYPE': 'SimpleCache'})
SearchCache.init_app(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quickannotator.db'
os.environ['SPATIALITE_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/mod_spatialite.so'

db.app = app
db.init_app(app)
models = [Project, Image, AnnotationClass, Notification, Tile, Setting]
with app.app_context():
    event.listen(db.engine, 'connect', load_spatialite)
    db.drop_all()
    db.metadata.create_all(bind=db.engine, tables=[item.__table__ for item in models])

    create_annotation_table(db, 1, 1, "gt")

/tmp/ipykernel_160875/2472280254.py:55: SAWarning: Table 'annotation' already exists within the given MetaData - not copying.
  table = Annotation.__table__.to_metadata(db.metadata, name=table_name)


In [85]:
path = "data/images/1/TCGA-23-2072-01Z-00-DX1.478243FF-BFF0-48A4-ADEA-DE789331A50E.svs"
slide = large_image.open(path)

In [86]:
insert_project(app, db, 
               name="example_project", 
               description="test", 
               is_dataset_large=False)

insert_image(app, db, 
             project_id=1, 
             name=os.path.basename(path), 
             path=path, 
             height=slide.sizeY, 
             width=slide.sizeX, 
             dz_tilesize=slide.tileWidth,
             embedding_coord="POINT(1 1)", 
             group_id=1, 
             split=1)

insert_annotation_class(app, db,
                        project_id=None,
                        name="Tissue Mask",
                        color="black",
                        magnification=None,
                        patchsize=None,
                        tilesize=None,
                        dl_model_objectref=None)



In [87]:
def insert_annotation(app, db, image_id, annotation_class_id, gtpred):
    with app.app_context():
        table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
        annotation = {"centroid" : "POINT(1 1)",
                                "area" : 1000,
                                "polygon" : "MULTIPOLYGON(((0 0, 0 1, 1 1, 1 0, 0 0)))",
                                "custom_metrics" : {"iou": 0.5}
        }
        
        
        
        table = Table(table_name, db.metadata, autoload_with=db.engine)
        stmt = insert(table).values(annotation)
        db.session.execute(stmt)
        db.session.commit()

In [88]:
insert_annotation(app, db, 1, 1, "gt")

In [89]:
table_name = f"1_1_gt_annotation"



with app.app_context():
    table = Table(table_name, db.metadata, autoload_with=db.engine)
    stmt = table.select()
    result = db.session.execute(stmt)



In [90]:
result.fetchall()

[(1, <WKBElement at 0x796de9e7e170; 0101000020ffffffff000000000000f03f000000000000f03f>, 1000.0, <WKBElement at 0x796de9e7cb50; 0106000020ffffffff0100000001030000000100000005000000000000000000000000000000000000000000000000000000000000000000f03f000000000000f03f000000000000f03f000000000000f03f000000000000000000000000000000000000000000000000>, {'iou': 0.5}, datetime.datetime(2024, 10, 18, 13, 26, 52))]